# Project: Data Preparation & Inputs


# 1. z-normalization
Each channel of raw data, x, is normalized individually as z = (x−µ)/σ, where µ and α are the mean and standard deviation of vector x

In [0]:
def zNorm(data):
  #for i in range(len(train_x_ts_data)):
  Sums = [];
  Avgs = [];
  Variances = [];
  SDevs = [];
  NumEles=0;

  for x in range(76):
    Sums.append(0);
    Avgs.append(0);
    Variances.append(0);
    SDevs.append(0);

  #data = [[[1,2,3,4],[76,55,44,21]],[[93,46,102,75],[74,33,51,935]],[[0,214,241,122],[150,151,251,500]]]


  #for each of 40 trials
  for TrialNum in range(len(data)):
    trial=data[TrialNum];
    #for time step 1 to 54
    for TimeStepNum in range(len(trial)):
      timeStep=trial[TimeStepNum];
      NumEles+=1;
      #for feature 1 to 76
      for FeatureNum in range(len(timeStep)):
        #add to sum
        Sums[FeatureNum]+=((float)(timeStep[FeatureNum])); #https://stackoverflow.com/questions/379906/how-do-i-parse-a-string-to-a-float-or-int-in-python

  for x in range (len(Sums)):
    Avgs[x]=Sums[x]/NumEles;

  #for each of 40 trials
  for TrialNum in range(len(data)):
    trial=data[TrialNum];
    #for time step 1 to 54
    for TimeStepNum in range(len(trial)):
      timeStep=trial[TimeStepNum];
      #for feature 1 to 76
      for FeatureNum in range(len(timeStep)):
        #add to devs
        Variances[FeatureNum]+=(((float)(timeStep[FeatureNum]))-Avgs[FeatureNum])*(((float)(timeStep[FeatureNum]))-Avgs[FeatureNum]);

  #divide by lens
  for x in range (len(Variances)):
    SDevs[x]=np.sqrt(Variances[x]/NumEles);  #https://docs.scipy.org/doc/numpy/reference/generated/numpy.sqrt.html
  newData=[];
  for x in range(len(data)):
    newData.append(data[x]); #https://stackoverflow.com/questions/6431973/how-to-copy-data-from-a-numpy-array-to-another

  #for each of 40 trials
  for TrialNum in range(len(data)):
    trial=newData[TrialNum];
    #for time step 1 to 54
    for TimeStepNum in range(len(trial)):
      timeStep=trial[TimeStepNum];
      #for feature 1 to 76
      for FeatureNum in range(len(timeStep)):
        #normalize as you're supposed to
        timeStep[FeatureNum] = (((float)(timeStep[FeatureNum])) - Avgs[FeatureNum])/SDevs[FeatureNum]
    return newData;

# 2. Data augmentation with a sliding window

## jigDataSplit

**
** Splits a 2D numpy array of the form [[Data],[Classification]] into a 2D array of [[Windows],[Classification]] 

@parameter train_x: the z-normalized raw data

@parameter train_y: the original classifications (0,1,2)

@parameter stepSize: the amount we shift the window by each time


@parameter windowSize: the size of the window that we want for the time




In [0]:
def jigDataSplit(train_x, train_y, stepSize, windowSize):
  train_x_new = []
  train_y_new = []
  #n = count of new trials
  n = 0
  #for each of the 40 trials
  for i in range(40):
    #m = the index of the row we are at within the current trial
    m = 0
    #check if we have reached the end of the rows for the trial
    while (m + windowSize < len(train_x[i])):
      #create a new trial of length windowSize
      new_trial = []
      for j in range(windowSize):
        new_trial.append(train_x[i][j])
      #add new trial to the new testing dataset
      train_x_new.append(new_trial)
      #classify the new trial with the pre-existing classification
      train_y_new.append(train_y[i])
      m = m + stepSize
      n = n + 1
    #add a trial for the rest of the data padded by zeros
    zero_padded_trial = []
    #add rest of original data
    for j in range(m, len(train_x[i])):
      zero_padded_trial.append(train_x[i][j])
    #add zeros padding
    for j in range (len(train_x[i]), m + windowSize):
      zero_padded_trial.append([0]*76)
    train_x_new.append(zero_padded_trial)
    train_y_new.append(train_y[i])
    n = n + 1
  samples_new = train_x_new
  for i in range(len(train_x_new)):
    samples_new[i] = np.array(train_x_new[i])
    print(samples_new[i].shape)
  samples_new = np.array(samples_new)
  classifications_new = np.array(train_y_new)
  return (samples_new, classifications_new)







##Split the data between master-side and patient-side manipulators

**
**
Split a 2D numpy array of the form [[Windows],[Classification]] into a list containing two 2D numpy arrays, one for master-side data and one for patient-side data

Data from each row of length 76 is split into two rows each of length 38


In [0]:
def splitMasterPatient(train_x, train_y, windowSize):
  new_train_x = []
  for window in train_x:
    window_split = np.hsplit(window, 2)
    new_train_x.append(window_split)
  return (new_train_x)
  new_train_y = []
  for classification in train_y:
    new_train_y.append([classicication, classification])
  master_data = []
  master_classes = []
  patient_data = []
  patient_classes = []
  for i in range(len(new_train_x)):
    master_data.append(new_train_x[i][0])
    master_classes.append(new_train_y[i][0])
    patient_data.append(new_train_x[i][1])
    patient_classes.append(new_train_y[i][1])
  master_data = np.array(master_data)
  master_classes = np.array(master_classes)
  patient_data = np.array(patient_data)
  patient_classes = np.array(patient_classes)
  return master_data + patient_data
  #return [[master_data, master_classes], [patient_data, patient_classes]]


#Main

In [5]:
import gzip
import pickle as pkl
import tensorflow as tf
import numpy as np
import scipy
from scipy import stats
from tempfile import TemporaryFile

data = np.load('test.npy')

# Convert numpy object back to dictionary
data = data[()]
# Split data for classification
train_x_ts_data = np.array(data['Data'])
train_y_classification = np.array(data['Class_Value'])
#print('Features', train_x_ts_data.shape)
#print('Class_Value', train_y_classification.shape)

#printing out what the shape of the entire input is
#samples = train_x_ts_data
#for i in range(len(train_x_ts_data)):
#   samples[i] = np.array(train_x_ts_data[i])
#   print(samples[i].shape)
#print('SAMPLE[0][0]',train_x_ts_data[0][0])
#print('Class[0]',train_y_classification[0])

#take data, Znorm
#split into windows
#save as npy object



#print('Trials[0][0]',data[0][0])
#print('Class[0]',train_y_classification[0])
#print('Avgs[0]',Avgs[0])
#print('NewData[0]',zNorm([[[1,2,3,4],[76,55,44,21]],[[93,46,102,75],[74,33,51,935]],[[0,214,241,122],[150,151,251,500]]]))
#print('SDevs[0]',SDevs[0])
#print('NewData:',newData)
#print(train_y_classification)


newX = zNorm(train_x_ts_data);
(x,y) = jigDataSplit(newX, train_y_classification, 30, 60)



#print('Features', x.shape)
#print('Class_Value', y.shape)
#print(y)

#save (x,y) tuple into .npy objects
outFile1 = TemporaryFile()
np.save('output1_x', x)

outFile2 = TemporaryFile()
np.save('output1_y', y)

#master = splitMasterPatient(x, y, 60)
#print(master)
#print(y)

#https://docs.scipy.org/doc/numpy/reference/generated/numpy.save.html


(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
Features (89, 60, 76)
Class_Value (89,)
[1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 

  Could not find a version that satisfies the requirement tensorflow.nn (from versions: )
No matching distribution found for tensorflow.nn


In [0]:
# Imports
import numpy as np
import tensorflow as tf
from time import time
from tensorflow.nn import softmax_cross_entropy_with_logits_v2 as categorical_cross_entropy
from tensorflow.contrib.layers import conv2d, max_pool2d, fully_connected, flatten, dropout
#import tensorflow.nn.softmax
from tensorflow.losses import softmax_cross_entropy
from tensorflow.train import AdamOptimizer

In [7]:
data_x = x
data_y =y 
train_x = np.array(data_x)
train_y = np.array(data_y)
#train_y = np.transpose(train_y)

#data = data[()]
#train_x = np.array(data['Features'])
#train_y = np.array(data['Class_Value'])
#print(train_x.shape)
#print(train_y.shape)


samples = train_x
for i in range(len(train_x)):
   samples[i] = np.array(train_x[i])
   #print(samples[i].shape)
#print(samples[2])


(89, 60, 76)
(89,)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
(60, 76)
[['0.15376700' '-0.05356000' '0.30664900' ... '-0.74965000' '-0.35224000'
  '-0.43486200']
 ['0.15392500' '-0.05260600' '0.30505000' ... '-0.86647100' '-0.62153000'
  '-0.44204700'

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-04-20 19:28:45--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.92.108.98, 34.206.253.53, 34.196.237.103, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.92.108.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.28M  29.1MB/s    in 0.5s    

2019-04-20 19:28:45 (29.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://24381b43.ngrok.io


In [0]:
'''
CNN based from "Deep LEarning with Convolutional Neural Network for Objective Skill Evaluation
in Robot-assisted Surgery" by Ziheng Wang and Ann Majewicz Fey
10 January 2018 
arXiv: 1806.05796v1
'''
tf.reset_default_graph()

class CNN_Wang_Fey(object):
  def __init__ (self, trainer):
    self.X = tf.placeholder(tf.float32, shape = [None, None, None])#60, 76]) # W x C where W is length and C is channels
    self.Y = tf.placeholder(tf.int64, shape= [None,])
    MSM_X = tf.slice(self.X, [0, 0, 0], [-1, 60, 38])
    PSM_X = tf.slice(self.X, [0, 0, 38], [-1, 60, 38])
    X_r = tf.reshape(PSM_X, [-1, 60, 38, 1]) 
    #Y_r = tf.reshape(self.Y, [-1, 1])
    Y_one_hot = tf.one_hot(self.Y, 3)
    C1 = tf.contrib.layers.conv2d(inputs=X_r, num_outputs=38, kernel_size=(2, 2), stride=1, data_format = 'NHWC')
    M1 = tf.contrib.layers.max_pool2d(inputs=C1, kernel_size=[2, 2], stride=2, data_format = 'NHWC')
    D1 = tf.contrib.layers.dropout(inputs=M1, keep_prob=0.8)
    C2 = tf.contrib.layers.conv2d(inputs=D1, num_outputs=76, kernel_size=(2, 2), stride=1, data_format = 'NHWC')
    M2 = tf.contrib.layers.max_pool2d(inputs=C2, kernel_size=[2, 2], stride=2, data_format = 'NHWC')
    D2 = tf.contrib.layers.dropout(inputs=M2, keep_prob=0.8)
    C3 = tf.contrib.layers.conv2d(inputs=D2, num_outputs=152, kernel_size=(2, 2), stride=1, data_format = 'NHWC')
    M3 = tf.contrib.layers.max_pool2d(inputs=C3, kernel_size=[2, 2], stride=2, data_format = 'NHWC')
    D2 = tf.contrib.layers.dropout(inputs=M3, keep_prob=0.8)
    
    F1 = fully_connected(inputs=tf.contrib.layers.flatten(D2), num_outputs=64, activation_fn = tf.nn.softmax)
    FD1 = tf.contrib.layers.dropout(inputs=F1, keep_prob=0.5)
    F2 = fully_connected(inputs=FD1, num_outputs = 32)
    FD2 = tf.contrib.layers.dropout(inputs=F2, keep_prob=0.5)
    self.Out = fully_connected(inputs=FD2, num_outputs = 3, activation_fn = tf.nn.softmax)
    
    tf.summary.histogram("softmax", self.Out)
    self.loss = tf.losses.softmax_cross_entropy(Y_one_hot, self.Out)
    tf.summary.scalar('loss', self.loss)
    local_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    self.gradients = tf.gradients(self.loss, local_vars)
    self.apply_grads = trainer.apply_gradients(zip(self.gradients, local_vars))
    
    self.accuracy =  tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(self.Out, 1), tf.argmax(Y_one_hot, 1)), 
            tf.float32))
    tf.summary.scalar('accuracy', self.accuracy)

  
opt = AdamOptimizer()
net = CNN_Wang_Fey(opt)


In [15]:

with tf.Session() as sess:
    merge = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter( './log', sess.graph)
    
    # Run the initializer
    sess.run(tf.global_variables_initializer())
  
   
    training_epochs = 10
    display_step = 1
    # Fit all training data
    for epoch in range(training_epochs):
      
      summary, _ = sess.run([merge, net.apply_grads], feed_dict={net.X: train_x, net.Y: train_y})
      train_writer.add_summary(summary, epoch)
        # Display logs per epoch step
      if (epoch+1) % display_step == 0:
        c = sess.run(net.loss, feed_dict={net.X: train_x, net.Y: train_y})
        #print("Epoch:", '%04d' % (epoch+1), "cost=", "{%f}" % c) 
        acc = sess.run(net.accuracy, feed_dict={net.X: train_x, net.Y: train_y})
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{%f}" % c, "accuracy=", "{%f}" % acc) 
            # \"W=", sess.run(W), "b=", sess.run(b))

    print("Optimization Finished!")

Epoch: 0001 cost= {1.096561} accuracy= {0.438202}
Epoch: 0002 cost= {1.094386} accuracy= {0.426966}
Epoch: 0003 cost= {1.090875} accuracy= {0.539326}
Epoch: 0004 cost= {1.087595} accuracy= {0.516854}
Epoch: 0005 cost= {1.087264} accuracy= {0.528090}
Epoch: 0006 cost= {1.081335} accuracy= {0.516854}
Epoch: 0007 cost= {1.082471} accuracy= {0.494382}
Epoch: 0008 cost= {1.074433} accuracy= {0.494382}
Epoch: 0009 cost= {1.066763} accuracy= {0.505618}
Epoch: 0010 cost= {1.080781} accuracy= {0.505618}
Optimization Finished!
